In [1]:
import time
import os
import sys
import warnings

from selenium import webdriver
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")

In [2]:
def web_driver(baseurl, browser='firefox'):
    if 'linux' in sys.platform.lower():
        if browser.lower() == 'firefox':
            driver = webdriver.Firefox()
        else:
            driver = webdriver.Chrome()
    else:
        path = os.path.join(os.getcwd(), 'driver')
        if browser.lower() == 'firefox':
            driver = webdriver.Firefox(path)
        else:
            driver = webdriver.Chrome(path)

    driver.get(baseurl)
    return driver


def para_phrase(input_text):
    '''
    Enter the text in textbox & captured the Paraphrase text
    '''
    input_area = driver.find_element(by='id', value='inputText')
    input_area.send_keys(input_text)

    submit_button = driver.find_element(By.CSS_SELECTOR,
                                        'div button.quillArticleBtn')
    submit_button.click()
    time.sleep(10)
    output = driver.find_element(By.CSS_SELECTOR,
                                 'span#editable-content-within-article').text

    return output


def split_index(full_para):
    '''
    Diveded paragraph into smaller sentences
    - There is a word limit 125
    - So I have allowed maximum 50 word [But you can change]
    '''
    index = [0]
    count = 0
    for i, sentence in enumerate(full_para):
        count += len(sentence.split())
        if count > 50:
            print(count - len(sentence.split()))
            count = 0
            index.append(i)
    if index[-1] + 1 != len(full_para):
        index.append(len(full_para))
    return index


def pipeline(sentences):
    '''
    Call para_phrase function (Retry twice as declared in count)
    '''
    global driver

    count = 2
    while count > 0:
        try:
            para = '.'.join(sentences)
            tran_text = para_phrase(para)
            driver.find_element(by='id', value='inputText').clear()
            return tran_text
        except:
            driver.close()
            url = 'https://quillbot.com/'
            driver = web_driver(url)
        count -= 1

    print("Error", sentences)


def full_para_process(full_para):
    '''
    Call the full process
    - Full para split into smaller sentences
    - Call para pipeline function
    - COllect result
    '''
    result = []
    full_para = full_para.split('.')

    index = split_index(full_para)
    for s, e in zip(index, index[1:]):
        tran_text = pipeline(full_para[s:e])

        if tran_text:
            result.append(tran_text)

    return result

## Open Browser

In [3]:
global driver

url = 'https://quillbot.com/'
driver = web_driver(url)

## Input the text & Call the function

In [4]:
para = '''A data engineer is responsible for collecting, managing, and converting 
raw data into information that can be interpreted by data scientists and business analysts. 
Data accessibility is their ultimate goal, which is to enable organizations to utilize data for performance 
evaluation and optimization.
'''

transformed_text = full_para_process(para)
transformed_text = ''.join(transformed_text)

In [5]:
print(transformed_text)

A data engineer is in charge of gathering, managing, and transforming unprocessed data into knowledge that data scientists and business analysts can understand.
Its ultimate objective is to make data accessible so that businesses can use it for performance analysis and improvement.


## Close the driver

In [6]:
driver.close()